In [2]:
# Code in file autograd/two_layer_net_autograd.py
import torch

device = torch.device('cpu')
# device = torch.device('cuda') # Uncomment this to run on GPU

In [3]:
import numpy as np

In [44]:
R_exps = [1.0, 0.9, 0.8, 0.7, 0.6, 0.5, 0.4, 0.3, 0.2, 0.1]
N = 200 # input dimension and number of training behaviors
M = N # output dimension

In [45]:
# Define nonlinear activation function
def f(x): # nonlinear conversion function to binary
    return x.sigmoid()#tanh().add(1).div(2)

In [5]:
# Create input data
x = torch.tensor(np.genfromtxt("x_{}.csv".format(N), delimiter=','), device=device).float()
y = torch.tensor(np.genfromtxt("y_{}.csv".format(N), delimiter=','), device=device).float()

In [1]:
# determine lrs_N200 

In [181]:
loss_all_lr = []
for learning_rate in [100, 300, 500]: # try different learning rates for different R
    R = int(N**0.7) # 
    # Create random Tensors for weights; setting requires_grad=True means that we
    # want to compute gradients for these Tensors during the backward pass.
    w1 = torch.randn(N, R, device=device, requires_grad=True).float()
    w2 = torch.randn(R, M, device=device, requires_grad=True).float()

    ## initialize tensor variables for bias terms 
    b1 = torch.randn(1, R, device=device, requires_grad=True).float()# bias for hidden layer
    b2 = torch.randn(1, M, device=device, requires_grad=True).float() # bias for output layer

    t = 0
    loss = 1000
    loss_all = []
    while t < 40000:
      # Forward pass: compute predicted y using operations on Tensors. Since w1 and
      # w2 have requires_grad=True, operations involving these Tensors will cause
      # PyTorch to build a computational graph, allowing automatic computation of
      # gradients. Since we are no longer implementing the backward pass by hand we
      # don't need to keep references to intermediate values.
      y_pred = f(f(x.mm(w1).add(b1)).mm(w2).add(b2))

      # Compute and print loss. Loss is a Tensor of shape (), and loss.item()
      # is a Python number giving its value.
      loss = (y_pred - y).pow(2).mean()
      loss_all.append(loss.item())
      # print(t, loss.item())

      # Use autograd to compute the backward pass. This call will compute the
      # gradient of loss with respect to all Tensors with requires_grad=True.
      # After this call w1.grad and w2.grad will be Tensors holding the gradient
      # of the loss with respect to w1 and w2 respectively.
      loss.backward()

      # Update weights using gradient descent. For this step we just want to mutate
      # the values of w1 and w2 in-place; we don't want to build up a computational
      # graph for the update steps, so we use the torch.no_grad() context manager
      # to prevent PyTorch from building a computational graph for the updates
      with torch.no_grad():
        w1 -= learning_rate * w1.grad
        w2 -= learning_rate * w2.grad
        b1 -= learning_rate * b1.grad
        b2 -= learning_rate * b2.grad

        # Manually zero the gradients after running the backward pass
        w1.grad.zero_()
        w2.grad.zero_()
        b1.grad.zero_()
        b2.grad.zero_()
        t = t + 1
    loss_all_lr.append(loss_all)

In [ ]:
import matplotlib
from matplotlib import pyplot as plt
plt.figure(figsize=(10,7))
for i in range(len(loss_all_lr)):
    plt.plot(np.log(np.array(loss_all_lr[i])))
plt.legend()
plt.xlabel('Iteration')
plt.ylabel('Loss')
#plt.ylim([0, 0.5])
plt.show()